In [1]:
import pandas as pd
import os

In [2]:
df = pd.read_csv("car-good.csv")

In [3]:
# 특징과 라벨 분리
X = df.drop('Class', axis = 1)
Y = df['Class']

In [4]:
# 학습 데이터와 평가 데이터 분리
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)

In [5]:
Train_Y.value_counts()

negative    630
positive     18
Name: Class, dtype: int64

In [6]:
# 문자 라벨을 숫자로 치환 
Train_Y.replace({"negative":-1, "positive":1}, inplace = True)
Test_Y.replace({"negative":-1, "positive":1}, inplace = True)

In [7]:
Train_X.head() # Buying, Maint, Lug_boot, safety 변수가 범주형 변수로 판단됨

,Buying,Maint,Doors,Persons,Lug_boot,Safety
557,med,med,2,4,big,high
595,med,low,2,2,small,med
529,med,high,4,2,big,med
681,low,vhigh,3,4,big,low
854,low,low,4,2,big,high


In [8]:
# 자세한 범주형 변수 판별 => 모든 변수가 범주형임을 확인
for col in Train_X.columns:
    print(col, len(Train_X[col].unique()))

Buying 4
Maint 4
Doors 3
Persons 2
Lug_boot 3
Safety 3


In [9]:
# 더미화를 이용한 범주 변수 처리
# feature_engine는 0.6.1 버전을 설치해야 한다

In [10]:
Train_X = Train_X.astype(str) # 모든 변수가 범주이므로, 더미화를 위해 전부 string 타입으로 변환

In [12]:
# variables 에 더미화 대상이 되는 범주형 변수의 이름 목록을 지정한다
# drop_last에서 한 범주변수로부터 만드는 더미 변수 중에 마지막 더미 변수를 제거할지 설정한다
from feature_engine.categorical_encoders import OneHotCategoricalEncoder as OHE
dummy_model = OHE(variables = Train_X.columns.tolist(),
                 drop_last = True)

dummy_model.fit(Train_X)

d_Train_X = dummy_model.transform(Train_X)
d_Test_X = dummy_model.transform(Test_X)

In [13]:
d_Train_X.head()

,Buying_med,Buying_low,Buying_high,Maint_med,Maint_low,Maint_high,Doors_2,Doors_4,Persons_4,Lug_boot_big,Lug_boot_small,Safety_high,Safety_med
557,1,0,0,1,0,0,1,0,1,1,0,1,0
595,1,0,0,0,1,0,1,0,0,0,1,0,1
529,1,0,0,0,0,1,0,1,0,1,0,0,1
681,0,1,0,0,0,0,0,0,1,1,0,0,0
854,0,1,0,0,1,0,0,1,0,1,0,1,0


In [14]:
# 더미화를 한 뒤의 모델 테스트
from sklearn.neighbors import KNeighborsClassifier as KNN
model = KNN().fit(d_Train_X, Train_Y)
pred_Y = model.predict(d_Test_X)

from sklearn.metrics import f1_score
f1_score(Test_Y, pred_Y)

0.0

In [15]:
# 연속형 변수로 치환

In [16]:
Train_df = pd.concat([Train_X, Train_Y], axis = 1)
for col in Train_X.columns: # 보통은 범주 변수만 순회
    # col에 따른 Class의 평균을 나타내는 사전 (replace를 쓰기 위해, 사전으로 만듦)
    # key 가 컬럼이고 값이 평균값이 된다
    temp_dict = Train_df.groupby(col)['Class'].mean().to_dict() 
    Train_df[col] = Train_df[col].replace(temp_dict) # 변수 치환
    # 테스트 데이터도 같이 치환해줘야 함 (나중에 활용하기 위해서는 저장도 필요)
    Test_X[col] = Test_X[col].astype(str).replace(temp_dict) 

<ipython-input-16-1de6141a08ba>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test_X[col] = Test_X[col].astype(str).replace(temp_dict)


In [17]:
Train_df.head()

,Buying,Maint,Doors,Persons,Lug_boot,Safety,Class
557,-0.949686,-0.929412,-0.945455,-0.891892,-0.962791,-0.896714,-1
595,-0.949686,-0.838926,-0.945455,-1.000000,-0.942308,-0.934884,-1
529,-0.949686,-1.000000,-0.943396,-1.000000,-0.962791,-0.934884,-1
681,-0.840000,-1.000000,-0.944444,-0.891892,-0.962791,-1.000000,-1
854,-0.840000,-0.838926,-0.943396,-1.000000,-0.962791,-0.896714,-1


In [18]:
Train_X = Train_df.drop('Class', axis = 1)
Train_Y = Train_df['Class']

In [19]:
# 치환한 뒤의 모델 테스트
model = KNN().fit(Train_X, Train_Y)
pred_Y = model.predict(Test_X)

f1_score(Test_Y, pred_Y)


# 라벨을 고려한 전처리이므로 더미화보다 좋은 결과가 나왔다 
# 차원도 줄고 성능 상에 이점이 있으나, 

0.4166666666666667